# Table of Contents
 <p><div class="lev1"><a href="#The-IPython-task-interface"><span class="toc-item-num">1&nbsp;&nbsp;</span>The IPython task interface</a></div><div class="lev2"><a href="#Starting-the-IPython-controller-and-engines"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Starting the IPython controller and engines</a></div><div class="lev2"><a href="#Creating-a-LoadBalancedView-instance"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Creating a LoadBalancedView instance</a></div><div class="lev2"><a href="#Quick-and-easy-parallelism"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Quick and easy parallelism</a></div><div class="lev3"><a href="#Parallel-map"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Parallel map</a></div><div class="lev3"><a href="#Parallel-function-decorator"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Parallel function decorator</a></div><div class="lev2"><a href="#Dependencies"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Dependencies</a></div><div class="lev3"><a href="#Functional-Dependencies"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Functional Dependencies</a></div><div class="lev4"><a href="#@require"><span class="toc-item-num">1.4.1.1&nbsp;&nbsp;</span>@require</a></div><div class="lev4"><a href="#@depend"><span class="toc-item-num">1.4.1.2&nbsp;&nbsp;</span>@depend</a></div><div class="lev3"><a href="#other-dependencies"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>other dependencies</a></div><div class="lev2"><a href="#Retries-and-Resubmit"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Retries and Resubmit</a></div><div class="lev3"><a href="#Retries"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Retries</a></div><div class="lev3"><a href="#Resubmit"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>Resubmit</a></div><div class="lev2"><a href="#Schedulers"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Schedulers</a></div><div class="lev2"><a href="#summary"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>summary</a></div>

# The IPython task interface


The task interface to the cluster presents the engines as a fault tolerant, dynamic load-balanced system of workers. Unlike the multiengine interface, in the task interface the user have no direct access to individual engines. By allowing the IPython scheduler to assign work, this interface is simultaneously simpler and more powerful.

Best of all, the user can use both of these interfaces running at the same time to take advantage of their respective strengths. When the user can break up the user’s work into segments that do not depend on previous execution, the task interface is ideal. But it also has more power and flexibility, allowing the user to guide the distribution of jobs, without having to assign tasks to engines explicitly.

## Starting the IPython controller and engines


To follow along with this tutorial, you will need to start the IPython controller and four IPython engines. The simplest way of doing this is to use the ipcluster command:

```
$ ipcluster start -n 4
```

## Creating a LoadBalancedView instance


The first step is to import the IPython ipyparallel module and then create a Client instance, and we will also be using a LoadBalancedView, here called lview:

In [ ]:
from ipyparallel import Client

In [ ]:
rc = Client(profile='ipython-task-interface')

For load-balanced execution, we will make use of a LoadBalancedView object, which can be constructed via the client’s load_balanced_view() method:

In [ ]:
# default load-balanced view
lview = rc.load_balanced_view()

## Quick and easy parallelism


In many cases, you simply want to apply a Python function to a sequence of objects, but in parallel. Like the multiengine interface, these can be implemented via the task interface. The exact same tools can perform these actions in load-balanced ways as well as multiplexed ways: a parallel version of map() and @parallel() function decorator. If one specifies the argument balanced=True, then they are dynamically load balanced. Thus, if the execution time per item varies significantly, you should use the versions in the task interface.

### Parallel map

To load-balance map(),simply use a LoadBalancedView:

In [ ]:
lview.block = True
serial_result = map(lambda x:x**10, range(32))
parallel_result = lview.map(lambda x:x**10, range(32))
list(serial_result)==parallel_result

### Parallel function decorator

Parallel functions are just like normal function, but they can be called on sequences and in parallel. The multiengine interface provides a decorator that turns any Python function into a parallel function:

In [ ]:
@lview.parallel()
def f(x):
    return 10.0*x**4

# this is done in parallel
f.map(range(32))

## Dependencies

Often, pure atomic load-balancing is too primitive for your work. In these cases, you may want to associate some kind of Dependency that describes when, where, or whether a task can be run.

### Functional Dependencies

Functional dependencies are used to determine whether a given engine is capable of running a particular task. This is implemented via a special Exception class, UnmetDependency, found in ipyparallel.error. Its use is very simple: if a task fails with an UnmetDependency exception, then the scheduler, instead of relaying the error up to the client like any other error, catches the error, and submits the task to a different engine. This will repeat indefinitely, and a task will never be submitted to a given engine a second time.

You can manually raise the UnmetDependency yourself, but IPython has provided some decorators for facilitating this behaviour.

There are two decorators and a class used for functional dependencies:

In [ ]:
from ipyparallel import depend, require, dependent

#### @require

The simplest sort of dependency is requiring that a Python module is available. The @require decorator lets you define a function that will only run on engines where names you specify are importable:

In [ ]:
@require('numpy', 'zmq')
def myfunc():
    return dostuff()

Now, any time you apply myfunc(), the task will only run on a machine that has numpy and pyzmq available, and when myfunc() is called, numpy and zmq will be imported. You can also require specific objects, not just module names:

In [ ]:
def foo(a):
    return a*a

@require(foo)
def bar(b):
    return foo(b)

@require(bar)
def baz(c, d):
    return bar(c) - bar(d)

In [ ]:
lview.apply_sync(baz, 4, 5)

#### @depend

The @depend decorator lets you decorate any function with any other function to evaluate the dependency. The dependency function will be called at the start of the task, and if it returns False, then the dependency will be considered unmet, and the task will be assigned to another engine. If the dependency returns anything other than ``False``, the rest of the task will continue.

In [ ]:
def platform_specific(plat):
    import sys
    return sys.platform == plat

@depend(platform_specific, 'darwin')
def mactask():
    do_mac_stuff()

@depend(platform_specific, 'nt')
def wintask(win):
    do_windows_stuff()

### other dependencies

Other dependencies are outside the scope of this tutorial (because you probably won't need them), these include: graph and impossible dependencies.

## Retries and Resubmit

### Retries

Another flag for tasks is retries. This is an integer, specifying how many times a task should be resubmitted after failure. This is useful for tasks that should still run if their engine was shutdown, or may have some statistical chance of failing. The default is to not retry tasks.

### Resubmit

Sometimes you may want to re-run a task. This could be because it failed for some reason, and you have fixed the error, or because you want to restore the cluster to an interrupted state. For this, the Client has a rc.resubmit() method. This simply takes one or more msg_ids, and returns an AsyncHubResult for the result(s). You cannot resubmit a task that is pending - only those that have finished, either successful or unsuccessful.

## Schedulers

There are a variety of valid ways to determine where jobs should be assigned in a load-balancing situation. In IPython, we support several standard schemes, and even make it easy to define your own. The scheme can be selected via the scheme argument to ipcontroller, or in the TaskScheduler.schemename attribute of a controller config object.

By default the leastload scheme is used: always assign tasks to the engine with the fewest outstanding tasks.

## summary

The LoadBalancedView has many more powerful features that allow quite a bit of flexibility in how tasks are defined and run. The next places to look are in the following classes:

- LoadBalancedView
- AsyncResult
- apply()
- dependency

The following is an overview of how to use these classes together:

- Create a Client and LoadBalancedView
- Define some functions to be run as tasks
- Submit your tasks to using the apply() method of your LoadBalancedView instance.
- Use Client.get_result() to get the results of the tasks, or use the AsyncResult.get() method of the results to wait for and then receive the results.